In [1]:
# import libraries

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()


········


In [3]:
# Load a text file

text_loader = TextLoader("./state_of_the_union.txt")
docs = text_loader.load()

In [4]:
#chunk/split text
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

# save vector embeddings in a database
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [5]:
# find relevant documents
query = "What did the president say about Ketanji Brown Jackson"
relevant_docs = db.similarity_search(query)
print(relevant_docs[0].page_content)

# Note this is NOT querying a chat model - only finding the relevant info

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [15]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)
custom_rag_prompt

PromptTemplate(input_variables=['context', 'question'], template='Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nAlways say "thanks for asking!" at the end of the answer.\n\n{context}\n\nQuestion: {question}\n\nHelpful Answer:')

In [ ]:

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(query)

In [19]:
# Load a PDF

pdf_loader = PyPDFLoader("./prompt_engineering.pdf")
pdf_pages = pdf_loader.load_and_split()

In [20]:
# retrieve info from documents

faiss_index = FAISS.from_documents(pdf_pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("What are the different categories of prompts?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

3: TABLE I
CLASSIFYING PROMPT PATTERNS
Pattern Category Prompt Pattern
Input Semantics Meta Language Creation
Output Output Automater
Customization Persona
Visualization Generator
Recipe
Template
Error Identiﬁcation Fact Check List
Reﬂection
Prompt Question Reﬁnement
Improvement Alternative Approaches

2: textual statements approach is that it is intentionally int uitive
to users. In particular, we expect users will understand how to
express and adapt the statements in a contextually appropri ate
way for their domain. Moreover, since the underlying ideas o f
the prompt are captured, these same ideas 


In [21]:
query = "What are the different categories of prompts?"

retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 6})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke(query)

'The different categories of prompts are Input Semantics, Output Customization, Error Identification, Prompt Improvement, Interaction, and Context Control. Each category focuses on different aspects of prompt patterns in the context of conversational LLMs. Thanks for asking!'